In [2]:
import pandas as pd
import numpy as np


In [3]:
train = pd.read_csv('train.csv', index_col='Id')
test = pd.read_csv('test.csv', index_col='Id')
train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [4]:
#check for null values
train[train.isnull().any(axis=1)]
#no null values

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,


In [9]:
trainxxx = train.iloc[:, -1:].values
print(trainxxx)

[[5]
 [5]
 [2]
 ..., 
 [3]
 [3]
 [3]]


In [21]:
#start by running a randomforest classifier on the data as it is
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
random_forest_initial = RandomForestClassifier(n_estimators=300, bootstrap=True, oob_score=True)
random_forest_initial.fit(train.iloc[:,:-1].values, train.iloc[:, -1:].values)
print(random_forest_initial.score(train.iloc[:,:-1].values, train.iloc[:, -1:].values))

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


1.0


In [23]:
test_copy = test.copy()
test_copy['Cover_Type']=random_forest_initial.predict(test.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('RF-Initial.csv', header=True)

In [14]:
def getFeatureImportance(classifier, data):
    
    importance = classifier.feature_importances_
    indexes = np.argsort(importance)[::-1]
    for f in range(54): 
        print("%d. %s (%f)" % (f + 1, data.columns[indexes[f]], importance[indexes[f]]))
    #print(importance_random_forest)

In [54]:
getFeatureImportance(random_forest_initial, train)

1. Elevation (0.232304)
2. Horizontal_Distance_To_Roadways (0.093029)
3. Horizontal_Distance_To_Fire_Points (0.072519)
4. Horizontal_Distance_To_Hydrology (0.061933)
5. Vertical_Distance_To_Hydrology (0.054209)
6. Hillshade_9am (0.051130)
7. Aspect (0.049320)
8. Hillshade_3pm (0.046936)
9. Wilderness_Area4 (0.045151)
10. Hillshade_Noon (0.044262)
11. Slope (0.035768)
12. Soil_Type10 (0.022059)
13. Soil_Type38 (0.018309)
14. Soil_Type3 (0.018257)
15. Wilderness_Area1 (0.017585)
16. Soil_Type39 (0.016958)
17. Wilderness_Area3 (0.016561)
18. Soil_Type4 (0.012199)
19. Soil_Type40 (0.009733)
20. Soil_Type30 (0.008181)
21. Soil_Type2 (0.006246)
22. Soil_Type17 (0.006148)
23. Soil_Type13 (0.005940)
24. Soil_Type29 (0.005558)
25. Soil_Type22 (0.005400)
26. Soil_Type23 (0.004958)
27. Soil_Type32 (0.004881)
28. Soil_Type12 (0.004273)
29. Wilderness_Area2 (0.003618)
30. Soil_Type33 (0.003583)
31. Soil_Type11 (0.003405)
32. Soil_Type6 (0.002869)
33. Soil_Type24 (0.002430)
34. Soil_Type31 (0.002353

In [5]:
train_copy = train.copy()
train_with_removed_features = train_copy.drop(['Soil_Type7', 'Soil_Type15'], axis=1)

In [6]:
test_copy = test.copy()
test_with_removed_features = test_copy.drop(['Soil_Type7', 'Soil_Type15'], axis=1)
print(len(test_copy.columns))
print(len(test_with_removed_features.columns))

54
52


In [7]:
'''The sum and difference between two pair of attributes. The following pairs of
measures are considered, generating, therefore, 2x4 = 8 new attributes: (Elevation, Vertical distance to
Hydrology), (Horizontal distance to Hydrology, Horizontal distance to Fire Points), (Horizontal distance
to Hydrology, Horizontal distance to Roadways) and (Horizontal distance to Fire Points, Horizontal
distance to Roadways).'''
train_with_removed_features['Add_Elevation_VerticalDistanceToHydrology'] = train_with_removed_features['Elevation'] + train_with_removed_features['Vertical_Distance_To_Hydrology']
test_with_removed_features['Add_Elevation_VerticalDistanceToHydrology'] = test_with_removed_features['Elevation'] + test_with_removed_features['Vertical_Distance_To_Hydrology']
train_with_removed_features['Subtract_Elevation_VerticalDistanceToHydrology'] = train_with_removed_features['Elevation'] - train_with_removed_features['Vertical_Distance_To_Hydrology']
test_with_removed_features['Subtract_Elevation_VerticalDistanceToHydrology'] = test_with_removed_features['Elevation'] - test_with_removed_features['Vertical_Distance_To_Hydrology']

train_with_removed_features['Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points'] = train_with_removed_features['Horizontal_Distance_To_Hydrology'] + train_with_removed_features['Horizontal_Distance_To_Fire_Points']
test_with_removed_features['Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points'] = test_with_removed_features['Horizontal_Distance_To_Hydrology'] + test_with_removed_features['Horizontal_Distance_To_Fire_Points']
train_with_removed_features['Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points'] = train_with_removed_features['Horizontal_Distance_To_Hydrology'] - train_with_removed_features['Horizontal_Distance_To_Fire_Points']
test_with_removed_features['Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points'] = test_with_removed_features['Horizontal_Distance_To_Hydrology'] - test_with_removed_features['Horizontal_Distance_To_Fire_Points']

train_with_removed_features['Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways'] = train_with_removed_features['Horizontal_Distance_To_Hydrology'] + train_with_removed_features['Horizontal_Distance_To_Roadways']
test_with_removed_features['Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways'] = test_with_removed_features['Horizontal_Distance_To_Hydrology'] + test_with_removed_features['Horizontal_Distance_To_Roadways']
train_with_removed_features['Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways'] = train_with_removed_features['Horizontal_Distance_To_Hydrology'] - train_with_removed_features['Horizontal_Distance_To_Roadways']
test_with_removed_features['Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways'] = test_with_removed_features['Horizontal_Distance_To_Hydrology'] - test_with_removed_features['Horizontal_Distance_To_Roadways']

train_with_removed_features['Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways'] = train_with_removed_features['Horizontal_Distance_To_Fire_Points'] + train_with_removed_features['Horizontal_Distance_To_Roadways']
test_with_removed_features['Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways'] = test_with_removed_features['Horizontal_Distance_To_Fire_Points'] + test_with_removed_features['Horizontal_Distance_To_Roadways']
train_with_removed_features['Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways'] = train_with_removed_features['Horizontal_Distance_To_Fire_Points'] - train_with_removed_features['Horizontal_Distance_To_Roadways']
test_with_removed_features['Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways'] = test_with_removed_features['Horizontal_Distance_To_Fire_Points'] - test_with_removed_features['Horizontal_Distance_To_Roadways']

train_with_removed_features.head()
test_with_removed_features.head()

#still need to move the column cover type to last in train

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type39,Soil_Type40,Add_Elevation_VerticalDistanceToHydrology,Subtract_Elevation_VerticalDistanceToHydrology,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways
Id,,,,,,,,,,,,,,,,,,,,,
15121,2680,354,14,0,0,2684,196,214,156,6645,...,0,0,2680,2680,6645,-6645,2684,-2684,9329,3961
15122,2683,0,13,0,0,2654,201,216,152,6675,...,0,0,2683,2683,6675,-6675,2654,-2654,9329,4021
15123,2713,16,15,0,0,2980,206,208,137,6344,...,0,0,2713,2713,6344,-6344,2980,-2980,9324,3364
15124,2709,24,17,0,0,2950,208,201,125,6374,...,0,0,2709,2709,6374,-6374,2950,-2950,9324,3424
15125,2706,29,19,0,0,2920,210,195,115,6404,...,0,0,2706,2706,6404,-6404,2920,-2920,9324,3484


In [8]:
'''The euclidean distance applied to attributes Horizontal distance to Hydrology
and Vertical distance to Hydrology, generating 1 new attribute'''

#train_with_removed_features = train_with_removed_features.drop(['Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology'], axis=1)
#test_with_removed_features = test_with_removed_features.drop(['Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology'], axis=1)

train_with_removed_features['Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology'] = np.sqrt(train_with_removed_features['Horizontal_Distance_To_Hydrology']**2 + train_with_removed_features['Vertical_Distance_To_Hydrology']**2)
test_with_removed_features['Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology'] = np.sqrt(test_with_removed_features['Horizontal_Distance_To_Hydrology']**2 + test_with_removed_features['Vertical_Distance_To_Hydrology']**2)



In [9]:
train_with_removed_features.head()
#test_with_removed_features.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Cover_Type,Add_Elevation_VerticalDistanceToHydrology,Subtract_Elevation_VerticalDistanceToHydrology,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,5,2596,2596,6537,-6021,768,-252,6789,5769,258.000000
2,2590,56,2,212,-6,390,220,235,151,6225,...,5,2584,2596,6437,-6013,602,-178,6615,5835,212.084889
3,2804,139,9,268,65,3180,234,238,135,6121,...,2,2869,2739,6389,-5853,3448,-2912,9301,2941,275.769832
4,2785,155,18,242,118,3090,238,238,122,6211,...,2,2903,2667,6453,-5969,3332,-2848,9301,3121,269.235956
5,2595,45,2,153,-1,391,220,234,150,6172,...,5,2594,2596,6325,-6019,544,-238,6563,5781,153.003268


In [10]:
cols_at_end = ['Cover_Type']
train_with_removed_features = train_with_removed_features[[c for c in train_with_removed_features if c not in cols_at_end] + [c for c in cols_at_end if c in train_with_removed_features]]
train_with_removed_features.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Add_Elevation_VerticalDistanceToHydrology,Subtract_Elevation_VerticalDistanceToHydrology,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points,Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,2596,2596,6537,-6021,768,-252,6789,5769,258.000000,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,2584,2596,6437,-6013,602,-178,6615,5835,212.084889,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,2869,2739,6389,-5853,3448,-2912,9301,2941,275.769832,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,2903,2667,6453,-5969,3332,-2848,9301,3121,269.235956,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,2594,2596,6325,-6019,544,-238,6563,5781,153.003268,5


In [1]:
print(len(df.columns))
print(len(test_with_removed_features.columns))

NameError: name 'df' is not defined

In [86]:
random_forest_with_first_feature_engineering = RandomForestClassifier(n_estimators=500, bootstrap=True, oob_score=True)
random_forest_with_first_feature_engineering.fit(train_with_removed_features.iloc[:,:-1].values, train_with_removed_features.iloc[:, -1:].values)
print(random_forest_with_first_feature_engineering.score(train_with_removed_features.iloc[:,:-1].values, train_with_removed_features.iloc[:, -1:].values))






/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


1.0


In [89]:
test_copy = test_with_removed_features.copy()
test_copy['Cover_Type']=random_forest_with_first_feature_engineering.predict(test_with_removed_features.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('RF-First_Feature_Engineering.csv', header=True)

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
extra_tree_with_first_feature_engineering = ExtraTreesClassifier(n_estimators=500, bootstrap=True, oob_score=True)
extra_tree_with_first_feature_engineering.fit(train_with_removed_features.iloc[:,:-1].values, train_with_removed_features.iloc[:, -1:].values)
print(extra_tree_with_first_feature_engineering.score(train_with_removed_features.iloc[:,:-1].values, train_with_removed_features.iloc[:, -1:].values))



/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


1.0


In [12]:
test_copy = test.copy()
test_copy['Cover_Type']=extra_tree_with_first_feature_engineering.predict(test_with_removed_features.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('ET-First_Feature_Engineering.csv', header=True)

In [15]:
getFeatureImportance(extra_tree_with_first_feature_engineering, train_with_removed_features)

1. Subtract_Elevation_VerticalDistanceToHydrology (0.096716)
2. Elevation (0.090250)
3. Add_Elevation_VerticalDistanceToHydrology (0.069812)
4. Wilderness_Area4 (0.044596)
5. Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways (0.035879)
6. Horizontal_Distance_To_Roadways (0.034968)
7. Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways (0.034760)
8. Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways (0.033011)
9. Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways (0.029972)
10. Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points (0.029647)
11. Horizontal_Distance_To_Hydrology (0.029188)
12. Horizontal_Distance_To_Fire_Points (0.028925)
13. Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology (0.028897)
14. Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points (0.028166)
15. Hillshade_9am (0.027623)
16. Soil_Type10 (0.027421)
17. Aspect (0.

In [16]:
test_with_added_features_2 = test_with_removed_features.copy()
train_with_added_features_2 = train_with_removed_features.copy()
print(len(test_with_added_features_2.columns))
print(len(train_with_added_features_2.columns))

61
62


In [17]:
train_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Hydrology'] = train_with_added_features_2['Elevation'] + train_with_added_features_2['Horizontal_Distance_To_Hydrology']
test_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Hydrology'] = test_with_added_features_2['Elevation'] + test_with_added_features_2['Horizontal_Distance_To_Hydrology']
train_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Hydrology'] = train_with_added_features_2['Elevation'] - train_with_added_features_2['Horizontal_Distance_To_Hydrology']
test_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Hydrology'] = test_with_added_features_2['Elevation'] - test_with_added_features_2['Horizontal_Distance_To_Hydrology']

train_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Fire_Points'] = train_with_added_features_2['Elevation'] + train_with_added_features_2['Horizontal_Distance_To_Fire_Points']
test_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Fire_Points'] = test_with_added_features_2['Elevation'] + test_with_added_features_2['Horizontal_Distance_To_Fire_Points']
train_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Fire_Points'] = train_with_added_features_2['Elevation'] - train_with_added_features_2['Horizontal_Distance_To_Fire_Points']
test_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Fire_Points'] = test_with_added_features_2['Elevation'] - test_with_added_features_2['Horizontal_Distance_To_Fire_Points']

train_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Roadways'] = train_with_added_features_2['Elevation'] + train_with_added_features_2['Horizontal_Distance_To_Roadways']
test_with_added_features_2['Add_Elevation_Horizontal_Distance_To_Roadways'] = test_with_added_features_2['Elevation'] + test_with_added_features_2['Horizontal_Distance_To_Roadways']
train_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Roadways'] = train_with_added_features_2['Elevation'] - train_with_added_features_2['Horizontal_Distance_To_Roadways']
test_with_added_features_2['Subtract_Elevation_Horizontal_Distance_To_Roadways'] = test_with_added_features_2['Elevation'] - test_with_added_features_2['Horizontal_Distance_To_Roadways']

train_with_added_features_2.head()
test_with_added_features_2.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways,Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology,Add_Elevation_Horizontal_Distance_To_Hydrology,Subtract_Elevation_Horizontal_Distance_To_Hydrology,Add_Elevation_Horizontal_Distance_To_Fire_Points,Subtract_Elevation_Horizontal_Distance_To_Fire_Points,Add_Elevation_Horizontal_Distance_To_Roadways,Subtract_Elevation_Horizontal_Distance_To_Roadways
Id,,,,,,,,,,,,,,,,,,,,,
15121,2680,354,14,0,0,2684,196,214,156,6645,...,-2684,9329,3961,0.0,2680,2680,9325,-3965,5364,-4
15122,2683,0,13,0,0,2654,201,216,152,6675,...,-2654,9329,4021,0.0,2683,2683,9358,-3992,5337,29
15123,2713,16,15,0,0,2980,206,208,137,6344,...,-2980,9324,3364,0.0,2713,2713,9057,-3631,5693,-267
15124,2709,24,17,0,0,2950,208,201,125,6374,...,-2950,9324,3424,0.0,2709,2709,9083,-3665,5659,-241
15125,2706,29,19,0,0,2920,210,195,115,6404,...,-2920,9324,3484,0.0,2706,2706,9110,-3698,5626,-214


In [18]:
cols_at_end = ['Cover_Type']
train_with_added_features_2 = train_with_added_features_2[[c for c in train_with_added_features_2 if c not in cols_at_end] + [c for c in cols_at_end if c in train_with_added_features_2]]
train_with_added_features_2.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Subtract_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology,Add_Elevation_Horizontal_Distance_To_Hydrology,Subtract_Elevation_Horizontal_Distance_To_Hydrology,Add_Elevation_Horizontal_Distance_To_Fire_Points,Subtract_Elevation_Horizontal_Distance_To_Fire_Points,Add_Elevation_Horizontal_Distance_To_Roadways,Subtract_Elevation_Horizontal_Distance_To_Roadways,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,6789,5769,258.000000,2854,2338,8875,-3683,3106,2086,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,6615,5835,212.084889,2802,2378,8815,-3635,2980,2200,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,9301,2941,275.769832,3072,2536,8925,-3317,5984,-376,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,9301,3121,269.235956,3027,2543,8996,-3426,5875,-305,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,6563,5781,153.003268,2748,2442,8767,-3577,2986,2204,5


In [19]:

extra_tree_with_second_feature_engineering = ExtraTreesClassifier(n_estimators=500, bootstrap=True, oob_score=True)
extra_tree_with_second_feature_engineering.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(extra_tree_with_second_feature_engineering.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test.copy()
test_copy['Cover_Type']=extra_tree_with_second_feature_engineering.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('ET-Second_Feature_Engineering.csv', header=True)


/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


1.0


In [20]:
getFeatureImportance(extra_tree_with_second_feature_engineering, train_with_added_features_2)

1. Subtract_Elevation_VerticalDistanceToHydrology (0.080418)
2. Elevation (0.075485)
3. Add_Elevation_VerticalDistanceToHydrology (0.053041)
4. Subtract_Elevation_Horizontal_Distance_To_Hydrology (0.045743)
5. Wilderness_Area4 (0.040007)
6. Add_Elevation_Horizontal_Distance_To_Hydrology (0.039339)
7. Add_Elevation_Horizontal_Distance_To_Roadways (0.030028)
8. Add_Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways (0.025949)
9. Add_Elevation_Horizontal_Distance_To_Fire_Points (0.025801)
10. Horizontal_Distance_To_Roadways (0.025753)
11. Subtract_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways (0.025053)
12. Add_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways (0.024996)
13. Soil_Type10 (0.024950)
14. Subtract_Elevation_Horizontal_Distance_To_Roadways (0.024484)
15. Horizontal_Distance_To_Hydrology (0.024010)
16. Euclidean_Horizontal_Distance_To_Hydrology_Vertical_Distance_To_Hydrology (0.023838)
17. Aspect (0.022329)
18. Hillshade_9am (0.

In [23]:
from sklearn.ensemble import RandomForestClassifier
random_forest_with_second_feature_engineering = RandomForestClassifier(n_estimators=500, bootstrap=True, oob_score=True)
random_forest_with_second_feature_engineering.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(random_forest_with_second_feature_engineering.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test_with_removed_features.copy()
test_copy['Cover_Type']=random_forest_with_second_feature_engineering.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('RF-Second_Feature_Engineering.csv', header=True)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


1.0


In [24]:

random_forest_with_second_feature_engineering_tuning = RandomForestClassifier(max_depth=14, max_features=25, n_estimators=1000, bootstrap=True, oob_score=True)

random_forest_with_second_feature_engineering_tuning.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(random_forest_with_second_feature_engineering_tuning.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test_with_removed_features.copy()
test_copy['Cover_Type']=random_forest_with_second_feature_engineering_tuning.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('RF-Second_Feature_Engineering_Tuning.csv', header=True)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.974206349206


In [26]:
extra_tree_with_second_feature_engineering_1000 = ExtraTreesClassifier(n_estimators=1000, bootstrap=True, oob_score=True)
extra_tree_with_second_feature_engineering_1000.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(extra_tree_with_second_feature_engineering_1000.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test.copy()
test_copy['Cover_Type']=extra_tree_with_second_feature_engineering_1000.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('ET-Second_Feature_Engineering_1000.csv', header=True)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


1.0


In [30]:
from sklearn.ensemble import AdaBoostClassifier
ada_boosting_with_second_feature_engineering = AdaBoostClassifier(n_estimators=500)
ada_boosting_with_second_feature_engineering.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(ada_boosting_with_second_feature_engineering.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test.copy()
test_copy['Cover_Type']=ada_boosting_with_second_feature_engineering.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('ADA-Second_Feature_Engineering.csv', header=True)



/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.433267195767


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gradient_boosting_with_second_feature_engineering = GradientBoostingClassifier(n_estimators=100)
gradient_boosting_with_second_feature_engineering.fit(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values)
print(gradient_boosting_with_second_feature_engineering.score(train_with_added_features_2.iloc[:,:-1].values, train_with_added_features_2.iloc[:, -1:].values))


test_copy = test.copy()
test_copy['Cover_Type']=gradient_boosting_with_second_feature_engineering.predict(test_with_added_features_2.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('GBT-Second_Feature_Engineering.csv', header=True)

In [ ]:
#https://shankarmsy.github.io/posts/forest-cover-types.html
#http://www-di.inf.puc-rio.br/~milidiu/inf2979/anteprojetos/gribel.pdf
'''1) could use weighted scaling to train as cover types in test data are not equal which could cause issue
although not sure how it would impact the results in some other test set
2) could use a blending model where we use average of many other models
3) still to try xgboost, although adaboost and gradient boosting doesnt seem to work well
4) havent tried feature scaling'''

In [19]:
from sklearn.ensemble import ExtraTreesClassifier
extra_tree_initial = ExtraTreesClassifier(n_estimators=300, bootstrap=True, oob_score=True)
extra_tree_initial.fit(train.iloc[:,:-1].values, train.iloc[:, -1:].values)
print(extra_tree_initial.score(train.iloc[:,:-1].values, train.iloc[:, -1:].values))

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


1.0


In [20]:
test_copy = test.copy()
test_copy['Cover_Type']=extra_tree_initial.predict(test.values) 

test_copy=test_copy['Cover_Type'] 
test_copy.to_csv('ET-Initial.csv', header=True)